# Cleaning Data Exercises

In this exercise, we'll be returning to the American Community Survey data we used previously to measuring racial income inequality in the United States. In today's exercise, we'll be using it to measure the returns to education and how those returns vary by race and gender.




## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_cleaning.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}
```


### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

## Exercises

### Exercise 1

For these cleaning exercises, we'll return to the ACS data we've used before one last time. We'll be working with `US_ACS_2017_10pct_sample.dta`. Import the data (please use url for the autograder).

In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

acs = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)

### Exercise 2

For our exercises today, we'll focus on `age`, `sex`, `educ` (education), and `inctot` (total income). Subset your data to those variables, and quickly look at a sample of 10 rows.

In [2]:
acs_data = acs[["age", "sex", "educ", "inctot"]]
acs_data.sample(10)

,age,sex,educ,inctot
124460,57,male,grade 12,70000
235734,20,male,grade 12,0
134147,46,male,4 years of college,190000
101386,36,male,4 years of college,4600
42411,38,female,grade 12,7000
14502,70,male,4 years of college,176000
11819,70,female,grade 11,6700
264668,50,female,n/a or no schooling,3000
122698,49,male,grade 12,86700
189102,46,female,grade 12,52000


### Exercise 3

As before, all the values of `9999999` have the potential to cause us real problems, so replace all the values of `inctot` that are `9999999` with `np.nan`. 

In [3]:
acs_data["inctot"] = acs_data["inctot"].replace(to_replace=9999999, value=np.nan)

### Exercise 4

Attempt to calculate the average age of people in our data. What do you get? Why are you getting that error?

You *should* get an error in trying to answer this question, but **PLEASE LEAVE THE CODE THAT GENERATES THIS ERROR COMMENTED OUT SO YOUR NOTEBOOK WILL RUN IN THE AUTOGRADER**. 

Then talk about the error in a markdown cell.

In [4]:
# acs_data["age"].mean()

**TypeError: 'Categorical' with dtype category does not support reduction 'mean'**

### Exercise 5

We want to be able to calculate things using age, so we need it to be a numeric type. Check the current type of `age`, and look at all the values of `age` to figure out why it's categorical and not numeric. You should find two problematic categories. Store the values of these categories in `"ex5_age_young"` and `"ex5_age_old"` (once you find them, it should be clear which is which).

In [5]:
acs_data["age"].value_counts()

age
60    4950
54    4821
59    4776
56    4776
58    4734
      ... 
93     476
95     471
92     355
91     227
96      10
Name: count, Length: 97, dtype: int64

In [6]:
results = {}

results["ex5_age_young"] = "less than 1 year old"
results["ex5_age_old"] = "90 (90+ in 1980 and 1990)"

print(
    f"The first problematic category in the dataset is '{results['ex5_age_young']}'; the second is '{results['ex5_age_old']}.'"
)

The first problematic category in the dataset is 'less than 1 year old'; the second is '90 (90+ in 1980 and 1990).'


### Exercise 6

In order to convert `age` into a numeric variable, we need to replace those problematic entries with values that `pandas` can later convert into numbers. Pick appropriate substitutions for the existing values and replace the current values. 

**Hint 1:** Categorical variables act like strings, so you might want to use string methods! 

**Hint 2:** Remember that characters like parentheses, pluses, asterisks, etc. are special in Python strings, and you have to escape them if you want them to be interpreted literally!

**Hint 3:** Because the US Census has been conducted regularly for hundreds of years but exactly how the census has been conducted have occasionally changed, variables are sometimes coded in a way that might be interpreted in different ways for different census years. For example, hypothetically, one might write `90 (90+ in 1980 and 1990)` if the Censuses conducted in 1980 and 1990 used to top-code age at 90 (any values *over* 90 were just coded as 90), but more recent Censuses no longer top-coded age and recorded ages over 90 as the respondents actual age.

In [7]:
acs_data["age"] = acs_data["age"].cat.add_categories(["0"])
acs_data.loc[acs_data["age"] == results["ex5_age_young"], "age"] = "0"

acs_data["age"] = acs_data["age"].cat.add_categories(["90"])
acs_data.loc[acs_data["age"] == results["ex5_age_old"], "age"] = "90"

In [8]:
to_drop = acs_data["age"].isin([results["ex5_age_old"], results["ex5_age_young"]])

acs_data.drop(acs_data[to_drop].index, inplace=True)

acs_data["age"] = acs_data["age"].cat.remove_unused_categories()

### Exercise 7

Now convert age from a categorical to numeric. Calculate the average age amoung this group, and store it in `"ex7_avg_age"`.

In [9]:
acs_data.loc[~acs_data["age"].str.isnumeric(), "age"]

Series([], Name: age, dtype: category
Categories (97, object): ['1' < '2' < '3' < '4' ... '95' < '96' < '0' < '90'])

In [10]:
acs_data["age"] = acs_data["age"].astype("float")

In [11]:
results["ex7_avg_age"] = acs_data["age"].mean()

print(f"The average age in the converted dataset is {results['ex7_avg_age']:.2f}.")

The average age in the converted dataset is 41.30.


### Exercise 8

Let's now filter out anyone in our data whose age is less than 18. Note that before made `age` a numeric variable, we couldn't do this! Again, calculate the average age and this time store it in `"ex8_avg_age"`. 

Use this sample of people 18 and over for all subsequent exercises.

In [12]:
results["ex8_avg_age"] = acs_data[acs_data["age"] >= 18]["age"].mean()

print(f"The average age of adults in the dataset is {results['ex8_avg_age']:.2f}.")

The average age of adults in the dataset is 49.76.


### Exercise 9

Create an indicator variable for whether each person has *at least* a college Bachelor's degree called `college_degree`. Use this variable to calculate the number of people in the dataset with a college degree. You may assume that to get a college degree you need to complete at least 4 years of college. Save the result as `"ex9_num_college"`.

In [13]:
acs_data["educ"].value_counts()

educ
grade 12                     93133
4 years of college           47212
1 year of college            38779
5+ years of college          29801
nursery school to grade 4    24514
grade 5, 6, 7, or 8          21535
2 years of college           20757
n/a or no schooling          19562
grade 11                      8758
grade 10                      7818
grade 9                       7135
Name: count, dtype: int64

In [14]:
acs_data["college_degree"] = acs_data["educ"].isin(
    ["4 years of college", "5+ years of college"]
)

acs_data["college_degree"].value_counts()

college_degree
False    241991
True      77013
Name: count, dtype: int64

In [15]:
results["ex9_num_college"] = acs_data["college_degree"].sum()

print(
    f"The number of people with college degrees in the dataset is {results['ex9_num_college']:,}."
)

The number of people with college degrees in the dataset is 77,013.


### Exercise 10

Let's examine how the educational gender gap. Use `pd.crosstab` to create a cross-tabulation of `sex` and `college_degree`. `pd.crosstab` will give you the number of people who have each combination of `sex` and `college_degree` (so in this case, it will give us a 2x2 table with Male and Female as rows, and `college_degree` True and False as columns, or vice versa. 

In [16]:
pd.crosstab(acs_data["college_degree"], acs_data["sex"])

sex,male,female
college_degree,,
False,119684,122307
True,36181,40832


### Exercise 11

Counts are kind of hard to interpret. `pd.crosstab` can also normalize values to give percentages. Look at the `pd.crosstab` help file to figure out how to normalize the values in the table. Normalize them so that you get the share of men with and without college degree, and the share of women with and without college degrees.

Store the share (between 0 and 1) of men with college degrees in `"ex11_share_male_w_degrees"`, and the share of women with degrees in `"ex11_share_female_w_degrees"`.

In [17]:
pd.crosstab(acs_data["college_degree"], acs_data["sex"], normalize="columns")

sex,male,female
college_degree,,
False,0.76787,0.74971
True,0.23213,0.25029


In [18]:
results["ex11_share_male_w_degrees"] = 0.23213
results["ex11_share_female_w_degrees"] = 0.25029

print(
    f"The share of males with a college degrees is {results['ex11_share_male_w_degrees']:.2%};\n"
    f"The share of females with a college degrees is {results['ex11_share_female_w_degrees']:.2%}."
)

The share of males with a college degrees is 23.21%;
The share of females with a college degrees is 25.03%.


### Exercise 12

Now, let's recreate that table for people who are 40 and over and people under 40. Over time, what does this suggest about the absolute difference in the share of men and women earning college degrees? Has it gotten larger, stayed the same, or gotten smaller? Store your answer (either `"the absolute difference has increased"` or `"the absolute difference has decreased"`) in `"ex12_comparing"`.

In [19]:
acs_u40 = acs_data[acs_data["age"] < 40]
pd.crosstab(acs_u40["college_degree"], acs_u40["sex"], normalize="columns")

sex,male,female
college_degree,,
False,0.856868,0.810193
True,0.143132,0.189807


In [20]:
acs_40nover = acs_data[acs_data["age"] >= 40]
pd.crosstab(acs_40nover["college_degree"], acs_40nover["sex"], normalize="columns")

sex,male,female
college_degree,,
False,0.682123,0.699144
True,0.317877,0.300856


In [ ]:
diff_u40 = 0.189807 - 0.143132
diff_40nover = 0.317877 - 0.300856

if diff_40nover < diff_u40:
    results["ex12_comparing"] = "the absolute difference has decreased"
else:
    results["ex12_comparing"] = "the absolute difference has increased"

### Exercise 13

In words, what is causing the change noted in Exercise 12 (i.e., looking at the tables above, tell me a story about Men and Women's College attainment).

**The increase of the absolute difference in the share of men and women earning college degrees over time is due to women's educational attainment has increased relative to men (they've gone from being behind to being ahead).**

In [22]:
for k, v in results.items():
    print(f"{k}: {v}")

ex5_age_young: less than 1 year old
ex5_age_old: 90 (90+ in 1980 and 1990)
ex7_avg_age: 41.30384885455982
ex8_avg_age: 49.75769659413359
ex9_num_college: 77013
ex11_share_male_w_degrees: 0.23213
ex11_share_female_w_degrees: 0.25029
ex12_comparing: the absolute difference has increased


In [23]:
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}

## Want More Practice?

Calculate the educational racial gap in the United States for White Americans, Black Americans, Hispanic Americans, and other groups. 

Note that to do these calculations, you'll have to deal with the fact that unlike most Americans, the American Census Bureau treats "Hispanic" not as a racial category, but a linguistic one. As a result, the racial category "White" in `race` actually includes most Hispanic Americans. For this analysis, we wish to work with the mutually exclusive categories of "White, non-Hispanic", "White, Hispanic", "Black (Hispanic or non-Hispanic)", and a category for everyone else. 